In [1]:
# =========================================================================================
# 1. LIBRERÍAS
# =====================================================
library(chromote)
library(rvest)
library(dplyr)
library(stringr)
library(lubridate)

Warning message:
"package 'chromote' was built under R version 4.4.3"
Warning message:
"package 'rvest' was built under R version 4.4.3"
Warning message:
"package 'dplyr' was built under R version 4.4.3"

Adjuntando el paquete: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'lubridate' was built under R version 4.4.3"

Adjuntando el paquete: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union




In [2]:
# =====================================================
# 2. URL
# =====================================================
url <- "https://portalmx.infonavit.org.mx/wps/portal/infonavitmx/mx2/el-instituto/el-infonavit/infonavit_agenda_investigacion/investigacion_analisis/reportes_estadisticos/!ut/p/z1/pZFNDoIwEIXPwgk6LbTQJaEBK7FQiIDdGFaERNGF8fyKOyIWiLOb5Hvz8x4yqEFmaJ991z7629Be3v3JsDNLAHaRh1VSphy0yIqioBmOJEP1FMj8OAIdU1YeIAegBJl_9Lm7UR9UQQhacUVS5pJ9yNbpORceTgMgOVYAutJYSOEDBW-dHn5UCBvv_waMfXyNjG1F5vsLwGjRFJjJ0AqMIU2BmRQ-gM3mpUfv1-NYDfSyl53jvABnC7Dx/dz/d5/L2dBISEvZ0FBIS9nQSEh/"
base <- "https://portalmx.infonavit.org.mx"


In [3]:
# =====================================================
# 3. INICIAR SESIÓN CHROME
# =====================================================
b <- ChromoteSession$new()
b$Page$navigate(url)
Sys.sleep(10)  # esperar a que cargue  :D

$frameId
[1] "D37EC759A8BFE62A26E9BEF949BAD537"

$loaderId
[1] "2D9555A8341CF0E7DA298CF49FB9BA9D"

$isDownload
[1] FALSE

In [4]:
# =====================================================
# 4. FUNCIONES PARA EXTRAER SEMANAS Y FECHAS
# =====================================================
extraer_semanas <- function(doc_html) {
  anchors <- html_elements(doc_html, "a")
  df <- data.frame(
    texto = html_text2(anchors),
    href  = html_attr(anchors, "href"),
    stringsAsFactors = FALSE
  )
  
  # Solo links que comienzan con "Semana"
  semanas <- df %>% filter(str_detect(texto, "^Semana") & !is.na(href))
  
  # Convertir links relativos a absolutos
  semanas$href <- ifelse(str_starts(semanas$href, "/"), paste0(base, semanas$href), semanas$href)
  
  # Eliminar duplicados (por link y texto)
  semanas <- semanas %>% distinct(href, texto, .keep_all = TRUE)
  
  return(semanas)
}


In [5]:
extraer_fechas <- function(texto) {
  # Normalizar abreviaturas de meses
  texto <- str_replace_all(texto, "ene", "enero")
  texto <- str_replace_all(texto, "feb", "febrero")
  texto <- str_replace_all(texto, "mar", "marzo")
  texto <- str_replace_all(texto, "abr", "abril")
  texto <- str_replace_all(texto, "may", "mayo")
  texto <- str_replace_all(texto, "jun", "junio")
  texto <- str_replace_all(texto, "jul", "julio")
  texto <- str_replace_all(texto, "ago", "agosto")
  texto <- str_replace_all(texto, "sep", "septiembre")
  texto <- str_replace_all(texto, "oct", "octubre")
  texto <- str_replace_all(texto, "nov", "noviembre")
  texto <- str_replace_all(texto, "dic", "diciembre")
  
  # Extraer días
  dia_inicio <- as.numeric(str_extract(texto, "(?<=Semana )\\d+"))
  dia_fin <- as.numeric(str_extract(texto, "(?<= al | a )\\d+"))
  
  # Extraer meses
  mes_inicio <- str_extract(texto, "(?<=de )enero|(?<=de )febrero|(?<=de )marzo|(?<=de )abril|(?<=de )mayo|(?<=de )junio|(?<=de )julio|(?<=de )agosto|(?<=de )septiembre|(?<=de )octubre|(?<=de )noviembre|(?<=de )diciembre")
  mes_fin <- str_extract(texto, "(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)$")
  
  if(is.na(mes_fin)) mes_fin <- mes_inicio
  
  return(list(dia_inicio=dia_inicio, dia_fin=dia_fin, mes_inicio=mes_inicio, mes_fin=mes_fin))
}



In [6]:
# =====================================================
# 5. RECORRER AÑOS Y EXTRAER TODOS LOS REPORTES
# =====================================================
años <- c("2026","2025","2024","2023")
todos_los_reportes <- list()

for(año in años){
  # Click en la pestaña del año
  js <- sprintf("document.querySelector('div[aria-label=\"%s\"]').click()", año)
  b$Runtime$evaluate(js)
  Sys.sleep(2)
  
  # Leer HTML actualizado
  html <- b$Runtime$evaluate("document.documentElement.outerHTML")$result$value
  doc <- read_html(html)
  
  # Extraer reportes semanales
  semanas <- extraer_semanas(doc)
  todos_los_reportes[[año]] <- semanas
}


In [7]:

# =====================================================
# 6. COMBINAR TODOS LOS AÑOS
# =====================================================
semanas <- bind_rows(todos_los_reportes)

# =====================================================
# 7. EXTRAER DÍAS Y MESES
# =====================================================
fechas <- lapply(semanas$texto, extraer_fechas)
fechas_df <- do.call(rbind, lapply(fechas, as.data.frame))

semanas <- bind_cols(semanas, fechas_df)

# =====================================================
# 8. CONVERTIR MES A NÚMERO
# =====================================================
meses <- c(
  enero=1, febrero=2, marzo=3, abril=4,
  mayo=5, junio=6, julio=7, agosto=8,
  septiembre=9, octubre=10, noviembre=11, diciembre=12
)
semanas$mes_inicio_num <- meses[semanas$mes_inicio]
semanas$mes_fin_num    <- meses[semanas$mes_fin]

# =====================================================
# 9. INFERIR AÑO AUTOMÁTICO
# =====================================================
anio_actual <- year(Sys.Date())
mes_actual  <- month(Sys.Date())

semanas$anio_inicio <- ifelse(semanas$mes_inicio_num > mes_actual, anio_actual - 1, anio_actual)
semanas$anio_fin <- ifelse(semanas$mes_fin_num < semanas$mes_inicio_num, semanas$anio_inicio + 1, semanas$anio_inicio)


In [8]:
# =====================================================
# 10. CREAR FECHAS Y TABLA FINAL
# =====================================================
semanas <- semanas %>%
  mutate(
    fecha_inicio = make_date(anio_inicio, mes_inicio_num, dia_inicio),
    fecha_fin    = make_date(anio_fin, mes_fin_num, dia_fin),
    semana_iso   = isoweek(fecha_inicio)
  )

tabla_semanales <- semanas %>%
  filter(!is.na(fecha_inicio) & !is.na(href)) %>%
  select(
    Año = anio_inicio,
    Mes_Inicio = mes_inicio,
    Mes_Fin = mes_fin,
    Semana_ISO = semana_iso,
    Fecha_Inicio = fecha_inicio,
    Fecha_Fin = fecha_fin,
    Link = href
  ) %>%
  arrange(desc(Fecha_Inicio)) %>%
  as.data.frame()
tabla_semanales

Año,Mes_Inicio,Mes_Fin,Semana_ISO,Fecha_Inicio,Fecha_Fin,Link
<dbl>,<chr>,<chr>,<dbl>,<date>,<date>,<chr>
2026,febrero,febrero,9,2026-02-27,2026-02-03,https://portalmx.infonavit.org.mx/wps/wcm/connect/f6478850-0f54-4e00-9ff5-1ea818d50ddf/20230303_B03B_InformeSemanalPu%CC%81blico+27+febrero-3+marzo.pdf?MOD=AJPERES&ContentCache=NONE&CACHE=NONE&CVID=oCjJafy
2026,febrero,febrero,9,2026-02-27,2026-02-03,https://portalmx.infonavit.org.mx/wps/wcm/connect/f6478850-0f54-4e00-9ff5-1ea818d50ddf/20230303_B03B_InformeSemanalPu%CC%81blico+27+febrero-3+marzo.pdf?MOD=AJPERES&ContentCache=NONE&CACHE=NONE&CVID=oCjJafy
2026,febrero,febrero,9,2026-02-27,2026-02-03,https://portalmx.infonavit.org.mx/wps/wcm/connect/f6478850-0f54-4e00-9ff5-1ea818d50ddf/20230303_B03B_InformeSemanalPu%CC%81blico+27+febrero-3+marzo.pdf?MOD=AJPERES&ContentCache=NONE&CACHE=NONE&CVID=oCjJafy
2026,febrero,febrero,9,2026-02-27,2026-02-03,https://portalmx.infonavit.org.mx/wps/wcm/connect/f6478850-0f54-4e00-9ff5-1ea818d50ddf/20230303_B03B_InformeSemanalPu%CC%81blico+27+febrero-3+marzo.pdf?MOD=AJPERES&ContentCache=NONE&CACHE=NONE&CVID=oCjJafy
2026,febrero,febrero,9,2026-02-26,2026-02-01,https://portalmx.infonavit.org.mx/wps/wcm/connect/d635455a-9742-4ab0-b310-7b5c80bdf968/BoletinEconomicoSemanalPublico26feb-1mzo.pdf?MOD=AJPERES&ContentCache=NONE&CACHE=NONE&CVID=oUfhKmz
2026,febrero,febrero,9,2026-02-26,2026-02-01,https://portalmx.infonavit.org.mx/wps/wcm/connect/d635455a-9742-4ab0-b310-7b5c80bdf968/BoletinEconomicoSemanalPublico26feb-1mzo.pdf?MOD=AJPERES&ContentCache=NONE&CACHE=NONE&CVID=oUfhKmz
2026,febrero,febrero,9,2026-02-26,2026-02-01,https://portalmx.infonavit.org.mx/wps/wcm/connect/d635455a-9742-4ab0-b310-7b5c80bdf968/BoletinEconomicoSemanalPublico26feb-1mzo.pdf?MOD=AJPERES&ContentCache=NONE&CACHE=NONE&CVID=oUfhKmz
2026,febrero,febrero,9,2026-02-26,2026-02-01,https://portalmx.infonavit.org.mx/wps/wcm/connect/d635455a-9742-4ab0-b310-7b5c80bdf968/BoletinEconomicoSemanalPublico26feb-1mzo.pdf?MOD=AJPERES&ContentCache=NONE&CACHE=NONE&CVID=oUfhKmz
2026,febrero,febrero,9,2026-02-24,2026-02-28,https://portalmx.infonavit.org.mx/wps/wcm/connect/00054c26-6fa1-4095-b457-69fd1d8e4734/Semana+24+al+28+de+febrero.pdf?MOD=AJPERES&ContentCache=NONE&CACHE=NONE&CVID=pq8i962
